In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.1)

from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("{nara}의 수도는 어디인가요? 그리고 당신의 이름은 무엇인가요?")

prompt = template.format(nara="대한민국")

chat.predict(prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 AI 비서입니다."),
    ("ai", "안녕하세요! 저는 지영봇입니다.") ,
    ("human", "{nara}의 수도는 어디인가요? 그리고 당신의 이름은 무엇인가요?")
])

prompt = template.format_messages(nara="대한민국")

chat.predict_messages(prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 요리사입니다. 요리에 대한 레시피를 간단하게 제안합니다."),
    ("human", "나는 {요리}를 만들기를 원합니다.")
])

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 채식주의자 요리사입니다. 레시피를 채식주의에 맞게 다시 레시피를 간단하게 알려주세요."),
    ("human", "{레시피}")
])

chef_chain = chef_prompt | chat
veg_chanin = veg_chef_prompt | chat

final_chain = {"레시피":chef_chain} | veg_chanin

final_chain.invoke({"요리":"김치찌개"})

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples =[
    {
        "질문": "김치찌개 만드는 방법",
        "답변": "김치찌개를 만들기 위해서는 먼저 김치를 준비하고, 돼지고기와 두부를 넣고 끓입니다. 마지막으로 대파와 고춧가루를 넣어 맛을 조절합니다."
    },
    {
        "질문": "된장찌개 만드는 방법",
        "답변": "된장찌개를 만들기 위해서는 된장을 풀고, 감자와 호박, 두부를 넣고 끓입니다. 마지막으로 대파와 청양고추를 넣어 맛을 조절합니다."   
    }
]

example_prompt = PromptTemplate.from_template("Human:{질문}\nAI: {답변}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: {요리}를 만드는 방법",
    input_variables=["요리"]
)

chain = prompt | chat

chain.invoke({"요리":"미역국"})

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples =[
    {
        "요리": "김치찌개",
        "답변": "김치찌개를 만들기 위해서는 먼저 김치를 준비하고, 돼지고기와 두부를 넣고 끓입니다. 마지막으로 대파와 고춧가루를 넣어 맛을 조절합니다."
    },
    {
        "요리": "된장찌개",
        "답변": "된장찌개를 만들기 위해서는 된장을 풀고, 감자와 호박, 두부를 넣고 끓입니다. 마지막으로 대파와 청양고추를 넣어 맛을 조절합니다."   
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{요리} 만드는 방법은 뭐에요?"),
    ("ai", "{답변}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 요리사입니다. 요리에 대한 레시피를 간단하게 제안합니다."),
    example_prompt
    ("human", "{요리} 만드는 방법은 뭐에요?")
])
chain = final_prompt | chat

chain.invoke({"요리":"미역국"})

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples =[
    {
        "질문": "김치찌개 만드는 방법",
        "답변": "김치찌개를 만들기 위해서는 먼저 김치를 준비하고, 돼지고기와 두부를 넣고 끓입니다. 마지막으로 대파와 고춧가루를 넣어 맛을 조절합니다."
    },
    {
        "질문": "된장찌개 만드는 방법",
        "답변": "된장찌개를 만들기 위해서는 된장을 풀고, 감자와 호박, 두부를 넣고 끓입니다. 마지막으로 대파와 청양고추를 넣어 맛을 조절합니다."   
    }
]

example_prompt = PromptTemplate.from_template("Human:{질문}\nAI: {답변}")

example_selector = LengthBasedExampleSelector.from_examples(
    examples=examples,
    example_prompt=example_prompt,
    max_length=10,
)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: {요리}를 만드는 방법",
    input_variables=["요리"]
)

prompt.format(요리="김치찌개")


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples =[
    {
        "질문": "김치찌개 만드는 방법",
        "답변": "김치찌개를 만들기 위해서는 먼저 김치를 준비하고, 돼지고기와 두부를 넣고 끓입니다. 마지막으로 대파와 고춧가루를 넣어 맛을 조절합니다."
    },
    {
        "질문": "된장찌개 만드는 방법",
        "답변": "된장찌개를 만들기 위해서는 된장을 풀고, 감자와 호박, 두부를 넣고 끓입니다. 마지막으로 대파와 청양고추를 넣어 맛을 조절합니다."   
    }
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human:{질문}\nAI: {답변}")

example_selector = RandomExampleSelector(
    examples=examples
)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: {요리}를 만드는 방법",
    input_variables=["요리"]
)

prompt.format(요리="김치찌개")


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="대한민국")


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="대한민국")


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

#총 4개의 프롬프트를 만들거에요.
intro = PromptTemplate.from_template(
    """
    너는 role-playing 어시스턴트야. {charactor}를 흉내내는 AI 비서야.
    """
)

example = PromptTemplate.from_template(
    """이것은 너가 어떻게 말해야하는지에 대한 예제입니다.
    Human: {example_question}
    You : {example_answer}
    """
)
#AI 도우미가 우리의 텍스트를 완성해주는 start 예제
start = PromptTemplate.from_template(
    """
    start now!
    Human: {question}
    You:
    """
)
#이 모든것을 하나로 합친 prompt
final = PromptTemplate.from_template(
    """{intro}
    {example}
    {start}
    """
)

prompts =[
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

chain = full_prompt | chat
chain.invoke({
    "charactor": "해적",
    "example_question": "해적이 되려면 어떻게 해야하나요?",
    "example_answer": "크르릉..그것은 비밀이다..!!.",
    "question": "너가 좋아하는 음식은 뭐에요?"
})

In [ ]:
import langchain.chat_models import ChatOpenAI
import langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

chat = ChatOpenAI(temperature=0.1)

#첫번째 질문에 답변과 두번쨰 답변이 걸리는 시간 보기
chat.predict("이탈리안 파스타는 어떻게 만드는건가요?") #13초

chat.predict("이탈리안 파스타는 어떻게 만드는건가요?") #0.1초



In [ ]:
import langchain.chat_models import ChatOpenAI
import langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db")) #db 에 캐싱해놓음

chat = ChatOpenAI(temperature=0.1)

#첫번째 질문에 답변과 두번쨰 답변이 걸리는 시간 보기
chat.predict("이탈리안 파스타는 어떻게 만드는건가요?") #13초

chat.predict("이탈리안 파스타는 어떻게 만드는건가요?") #0.1초



In [ ]:
#챌린지예제
#영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
#LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
#예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples =[
    {
        "영화이름": "거룩한밤",
        "감독": "임대희",
        "주요 출연진": "마동석, 서현, 이다윗, 경수진, 정지소",
        "예산": "약 150억원",
        "흥행 수익": "2025년 5월 기준 베트남 30만 관객 돌파",
        "장르": "액션",
        "시놉시스": "악을 숭배하는 집단에 의해 혼란에 빠진 도시에서, 특별한 능력을 가진 어둠의 해결사 '거룩한 밤' 팀(바우, 샤론, 김군)이 의뢰를 받아 악의 무리를 처단하는 오컬트 액션. 신경정신과 의사 정원이 악마에 빙의된 동생 은서를 구해달라고 의뢰하면서 팀이 거대한 악에 맞서 싸우게 된다."
    },
    {
        "영화이름": "계시록",
        "감독": "연상호",
        "주요 출연진": "류준열, 신현빈, 신민재, 한지현",
        "예산": "약 1600억원",
        "흥행 수익": "2025년 4월 기준 넷플릭스 공개 직후 글로벌 1위 기록",
        "장르": "범죄",
        "시놉시스": "지방 소도시의 목사 민찬이 신의 계시를 받아 소녀 실종 사건의 범인을 쫓으면서, 형사 연희와 전과자 양래가 얽히고설키는 현실적 범죄 스릴러. 종교적 신념과 집착, 사회의 어두운 면을 조명한다."   
    },
    {
        "영화이름": "대홍수",
        "감독": "김병우",
        "주요 출연진": "김다미, 박해수, 이재웅",
        "예산": "비공개",
        "흥행 수익": "비공개",
        "장르": "재난",
        "시놉시스": "지구에 대홍수가 닥치며, 인공지능 개발자 안나와 인적자원 보안팀 희조가 물에 잠긴 아파트에서 생존을 위해 사투를 벌인다. 희조가 안나를 구하려는 이유와 그 배후의 진실이 점차 드러난다."   
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{영화제목} 에 대하여 영화 정보 알려주세요"),
    ("ai", "{답변}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화정보를 안내해주는 영화안내자입니다. 영화에 대한 정보를 간단하게 제안합니다."),
    example_prompt
    ("human", "{영화제목} 에 대하여 영화 정보 알려주세요")
])
chain = final_prompt | chat

chain.invoke({"영화제목":"도가니"})